<a href="https://colab.research.google.com/github/DJCordhose/transformers/blob/main/notebooks/Mixtral_8x7B_Instruct_HQQ_A100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mixtral 8x7B Instruct HQQ quantize on A100

* https://mistral.ai/news/mixtral-of-experts/
* https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1
* https://huggingface.co/mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bitgs8-metaoffload-HQQ
* https://huggingface.co/docs/transformers/v4.42.0/quantization/hqq

### Prerequisites
1. a Huggingface account and an
1. access tokens to put in below while logging in
1. patience: this will take 10-15 minutes to load libs and model and even generation will be much faster than T4 or L4, but still slow
   * takes around 1-2 secs per token generated
   * 64 tokens will take around 2 minutes to generate

### Why Mixtral
* explicitly tuned for European languages (like French, Italian, German and Spanish)
* performance close to GPT 3.5
* even quantized performance is good
* only uses fraction of parameters at a time
* theoretically able to unload parameters to CPU

### Comparing GPU microarchitectures
* T4/RTX 20: https://en.wikipedia.org/wiki/Turing_(microarchitecture)
* A100/RTX 30: https://en.wikipedia.org/wiki/Ampere_(microarchitecture)
* L4/L40/RTX 40: https://en.wikipedia.org/wiki/Ada_Lovelace_(microarchitecture)
* H100 - variant of RTX 40 consumer line, not available on Colab (yet?): https://en.wikipedia.org/wiki/Hopper_(microarchitecture)
* Future successor to both Hopper and Ada Lovelace: https://en.wikipedia.org/wiki/Blackwell_(microarchitecture)
* comparing GPUs: https://www.reddit.com/r/learnmachinelearning/comments/18gn1b2/choosing_the_right_gpu_for_your_workloads_a_dive/


In [1]:
!nvidia-smi

Sun Jul 14 11:23:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0              45W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [12]:
%%time

# this will take a few minutes, hqq_aten important for best performance
!pip install hqq hqq_aten -q

CPU times: user 40.2 ms, sys: 0 ns, total: 40.2 ms
Wall time: 5.72 s


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from IPython.display import Markdown

In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: read).


execute in a termial 'watch -n 0.5 nvidia-smi' to see the GPU usage and when the model is loaded onto it

In [6]:
%%time

import transformers
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer
from hqq.core.quantize import *

model_id = 'mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bitgs8-metaoffload-HQQ'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = HQQModelForCausalLM.from_quantized(model_id)

HQQLinear.set_backend(HQQBackend.ATEN_BACKPROP)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/773 [00:00<?, ?B/s]

qmodel.pt:   0%|          | 0.00/24.1G [00:00<?, ?B/s]

100%|██████████| 32/32 [00:11<00:00,  2.75it/s]

CPU times: user 1min 25s, sys: 54.6 s, total: 2min 20s
Wall time: 2min 28s


In [7]:
!nvidia-smi

Sun Jul 14 11:27:54 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              51W / 400W |  13275MiB / 40960MiB |      5%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [8]:
from threading import Thread

def chat_processor(chat, max_new_tokens=100, do_sample=True):
    tokenizer.use_default_system_prompt = False
    streamer = transformers.TextIteratorStreamer(tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True)

    generate_params = dict(
        tokenizer("<s> [INST] " + chat + " [/INST] ", return_tensors="pt").to('cuda'),
        streamer=streamer,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        top_p=0.90,
        top_k=50,
        temperature= 0.6,
        num_beams=1,
        repetition_penalty=1.2,
    )

    t = Thread(target=model.generate, kwargs=generate_params)
    t.start()
    outputs = []
    for text in streamer:
        outputs.append(text)
        print(text, end="", flush=True)

    return outputs

In [9]:
%%time

outputs = chat_processor("How do I build a car?", max_new_tokens=10, do_sample=False)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1. Gather resources and materials: Before youCPU times: user 15.8 s, sys: 1.63 s, total: 17.4 s
Wall time: 17.2 s


In [10]:
%%time

messages = [
    # Mistral models can not handle system promt roles
    {"role": "user", "content": "You are an English-speaking, competent advisor in the field of statutory health insurance. Answer consice, serious and formal. Answer this question: What is the best German statutory health insurance?"},
]
tokenizer.use_default_system_prompt = False

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    # will run faster because it will not use full 512 tokens
    max_new_tokens=512,
    # max_new_tokens=64,
    # max_new_tokens=2,
    eos_token_id=terminators,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=False,
    num_beams=1
)
response = outputs[0][input_ids.shape[-1]:]
Markdown(tokenizer.decode(response, skip_special_tokens=True))


CPU times: user 3min 13s, sys: 0 ns, total: 3min 13s
Wall time: 3min 12s


Determining the "best" German statutory health insurance can be subjective and dependent on individual needs and preferences. However, some of the top-rated German statutory health insurance funds according to independent studies and customer satisfaction surveys are:

1. Techniker Krankenkasse (TK)
2. AOK Bundesverband (AOK-BV)
3. DAK-Gesundheit
4. Barmer Ersatzkasse
5. Kaufmännische Krankenkasse KKH

These funds consistently rank high in terms of services, benefits, and customer satisfaction. It is recommended that you compare their offerings and choose the one that best suits your personal requirements and preferences.

In [11]:
!nvidia-smi

Sun Jul 14 11:31:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0             115W / 400W |  13665MiB / 40960MiB |     95%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--